In [1]:
%load_ext autoreload
%autoreload 2

Imports

In [3]:
import sys
import os
import numpy as np
import pandas as pd
from scripts.utils.db import engine
from scripts.agent_script import chatbot_interaction

### Loading the df

In [4]:
df = pd.read_sql("SELECT * FROM marketing_data", engine)
df.head(3)

,year,quarter,month,week,date,country,media_category,media_name,communication,campaign_category,product,campaign_name,revenue,cost,profit,roi,margin,quarter_number,month_number,month_name
0,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 10,411.522496,9778.544045,-9367.021548,-0.957916,-22.761870,3,8,August
1,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Tactical,Category 3,Product 1,Campaign 11,228.913948,5083.770000,-4854.856052,-0.954972,-21.208214,3,8,August
2,2020,2020 Q3,2020M08,35,2020-08-24,DK,online,Interscroller,Branding,Category 5,Product 1,Campaign 12,71.051079,294.960000,-223.908921,-0.759116,-3.151380,3,8,August


In [5]:
interaction_log = list()

# First test, checking if the total revenue for 2021 is the same as the one I calculate

In [ ]:
questions = ["Revenue for 2021", 'exit'] # i mean total revenue for 2021

total_revenue_2021 = round(df[df['year'] == 2021]['revenue'].sum(), 2)

interaction_log = chatbot_interaction(questions) # chatbot call with predefined questions

print(f"The tool used is: {interaction_log[0]['tools_used'][0]['tool_name']}")

assert round(float(interaction_log[0]['agent_response']), 2) == total_revenue_2021

interaction_log.clear()

User: Revenue for 2021


# Second test, checking the same idea but with follow up

In [ ]:
questions = ["Revenue for 2021", "same but for 2022", 'exit'] # i mean total revenue for 2021 and 2022

total_revenue_2022 = round(df[df['year'] == 2022]['revenue'].sum(), 2)

interaction_log = chatbot_interaction(questions)

print(f"The tool used is: {interaction_log[0]['tools_used'][0]['tool_name']} and {interaction_log[1]['tools_used'][0]['tool_name']}")

assert round(float(interaction_log[0]['agent_response']), 2) == total_revenue_2021
assert round(float(interaction_log[1]['agent_response']), 2) == total_revenue_2022

interaction_log.clear()

# Third test, should use the grouping tool

In [ ]:
# here i also calculate just the category and not its name
questions = ["Which media categories had the highest profit in Q2 2023?", "exit"]

interaction_log = chatbot_interaction(questions)
print(f"The tool used is: {interaction_log[0]['tools_used'][0]['tool_name']}")

q2_2023 = df[
    (df["year"] == 2023) &
    (df["quarter_number"] == 2)
]
profit_by_category = (
    q2_2023
    .groupby("media_category", as_index=False)["profit"]
    .sum()
    .rename(columns={"profit": "total_profit"})
    .sort_values("total_profit", ascending=False)
)

assert profit_by_category.iloc[0]['media_category'] == interaction_log[0]['agent_response']



# Checking if a plot is the same, looking at it's ticks

In [ ]:
questions = ["monthly revenue for 2023", "exit"]
df_plot = pd.read_csv("scripts/plots_output/cost_month_name_year___2023.csv")

expected_df = (
    df[df["year"] == 2023]
    .groupby(["month_number", "month_name"])["cost"]
    .sum()
    .reset_index()
    .sort_values("month_number")
    .reset_index(drop=True)
)

assert list(df_plot["month_name"]) == list(expected_df["month_name"])
assert np.allclose(df_plot["cost"], expected_df["cost"])
